In [ ]:
import cv2
import time
import numpy as np
import math 
%pip install mediapipie
import handtrackingmodule as htm
import os

In [ ]:
def set_volume(level):
    """Set system volume (0 to 100)."""
    os.system(f"osascript -e 'set volume output volume {level}'")

set_volume(50)  

In [ ]:
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)

# Function to Set Volume on macOS
def set_volume(volume):
    volume = int(volume)  # Convert to integer
    os.system(f"osascript -e 'set volume output volume {volume}'")

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img = detector.findHands(img)
    lmList = detector.findPosition(img)

    if len(lmList) != 0:
        x1, y1 = lmList[4][1], lmList[4][2]  # Thumb
        x2, y2 = lmList[8][1], lmList[8][2]  # Index Finger
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Center

        # Draw Circles and Line
        cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

        # Calculate Distance and Volume
        length = math.hypot(x2 - x1, y2 - y1)
        volume = np.interp(length, [20, 250], [0, 100])

        set_volume(volume)

        # Map Volume to Bar (Invert Scale)
        volBar = np.interp(volume, [0, 100], [400, 50])
        volPercentage = int(np.interp(volume, [0, 100], [0, 100]))

        # Draw Volume Bar
        cv2.rectangle(img, (50, 50), (85, 400), (0, 255, 0), 3)  # Bar Outline
        cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)  # Volume Level

        # Display Volume Percentage
        cv2.putText(img, f'{volPercentage}%', (40, 450), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)

        # Change Color if Fingers are Close
        if length < 50:
            cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)

    # Calculate FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    cv2.imshow("video", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

I0000 00:00:1742658718.434709 6815201 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1742658718.471340 7130093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742658718.486407 7130092 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
